Apparently keras ImageDataGenerator is often used in Standardizing/normalizing image data. See two sources and the doc for ImageDataGenerator

https://machinelearningmastery.com/how-to-normalize-center-and-standardize-images-with-the-imagedatagenerator-in-keras/

https://www.geeksforgeeks.org/how-to-normalize-center-and-standardize-image-pixels-in-keras/

Doc ImageDataGenerator
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

Some explanation:

ImageDataGenerator comes from keras, which is a python library.
It can scale the model during the training phase.
It can also be used for augmentation.

In [2]:
%pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.7 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 10.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached libclang-16.0.0-py2.py3-none-macosx_11_0_arm64.whl (24.3 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.23.3-cp37-abi3-macosx_10_9_universal2.whl (400 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 20.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.4 MB/s eta 0:00:003

In [10]:
# import required modules
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator
 
# loading the image dataset
(trainX, trainY), (testX, testY) = mnist.load_data()


 
# # reshaping the dataset to have a single channel
width, height, channels = trainX.shape[1], trainX.shape[2], 1
trainX_shaped = trainX.reshape((trainX.shape[0], width, height, channels))
# testX = testX.reshape((testX.shape[0], width, height, channels))
# trainY = to_categorical(trainY)
# testY = to_categorical(testY)
 
# # confirming scale of pixel values
# print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
# print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
 
# # creating the image data generator [1.0/255.0 = 0.00392156862]
# datagen = ImageDataGenerator(rescale=1.0/255.0)
 
# # preparing an iterator for scaling images
# train_iterator = datagen.flow(trainX, trainY, batch_size=64)
# test_iterator = datagen.flow(testX, testY, batch_size=64)
# print('Batches train=%d, test=%d' % (len(train_iterator),
#                                      len(test_iterator)))
# # confirming- the scaling works
# batchX, batchy = train_iterator.next()
# print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(),
#                                               batchX.max()))

In [11]:
trainX[0].shape
trainX_shaped[0].shape

(28, 28, 1)

the keras.fit_generator() is used to implement the scaling during training the model.

First the ImageDataGenerator is called with certain parameters and stored in VARIABLE
Then it is used in VARIABLE.flow() to prepare an iterator, the training sets as well as the batch size is given as input.

1) Creating the model layers
2) Using an optimizer on the model
3) model.fit_generator() <- here the previously created iterator is given as input!

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0, 
    featurewise_center=False,  # set input mean to 0 over the dataset 
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.2, # Randomly zoom image 
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [ ]:
# Create an ImageDataGenerator and input the chosen scaling choices (also augmentation is possible)
datagen = ImageDataGenerator()

# if needed (depends on scaling method), calculate for the whole training data set the statistics using the .fit() function. Later on this can be applied to test and validation data set.
datagen.fit(X_train)

# A neural network model can be fitted with the data generator by using .flow() . It retrieves an iterator which returns batches of data and passes it to the fit_generator() function.

# creating the iterator
train_iterator = datagen.flow(X_train, y_train)

# optional: creating an iterator for the validation data set (only used if a validation data set is present)
val_iterator = datagen.flow(X_val, y_val)

# fitting the model, also other parameters should be passed here
model.fit_generator(train_iterator)

# The model can also be evaluated:
test_iterator = datagen.flow(X_test, y_test)
loss = model.evaluate_generator(test_iterator)

In [ ]:
https://machinelearningmastery.com/how-to-normalize-center-and-standardize-images-with-the-imagedatagenerator-in-keras/